# Think Green Investing 

In [157]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import panel as pn



pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import plotly.graph_objects as go
import plotly.offline as pyo
from finta import TA
import talib as ta 

%matplotlib inline

## Data Preparation

In [158]:
#initializing alpaca trade api
load_dotenv()
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
alpaca=tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version='v2')

In [159]:
#function that pulls tickers daily closing prices in the given time period using alpaca trade api, 
#intialization of the api is required before using.
#It returns a dataframe with the tickers, and closing prices as two level column structured datafram with index defined as date  

def get_tickers_data(tickers, start_date, end_date, timeframe):
    start_date = pd.Timestamp(start_date, tz = "America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz = "America/New_York").isoformat()
    tickers_data_df = alpaca.get_bars(tickers, timeframe,start_date , end_date, adjustment = 'raw').df
    tickers_data_df.index = tickers_data_df.index.date
    tickers_data_df = tickers_data_df[['symbol', 'open', 'high', 'low','close']]
    tickers_data_list=[]
    for ticker in tickers:
        tickers_data_list.append(tickers_data_df[tickers_data_df['symbol']==ticker].drop('symbol', axis=1))
        
    clean_tickers_data = pd.concat(tickers_data_list,axis=1, keys=tickers)
    #clean_tickers_data.columns=clean_tickers_data.columns.droplevel(-1)
    return clean_tickers_data

In [160]:
#defining the stocks in the solar energy portfolio form top performers in 2021 using Motley fool and Investopedia. The companies are:
#1.First Solar (FSLR)
#2.Solar Edge Technologies (SEDG)
#3.Daqo New Energy Corp. (DQ)
#4.Canadian Solar INC (CSIQ)
#5.Algonquin Power & Utilities Corp (AQN)

# Initialize data

# Set the ticker information
portfolio = ['FSLR' , 'SEDG', 'DQ', 'CSIQ', 'AQN']
indices = ['SPY']
timeframe='1Day'
tickers = portfolio + indices

### Set the Period: from Jan 2019 to June 2022

In [161]:
#Setting up period and pulling data and cleaning it. 
start_date = '2019-01-01'
end_date = '2022-06-15'

tickers_data = get_tickers_data(tickers, start_date, end_date, timeframe) 
tickers_data.dropna(inplace = True)

In [162]:
#function to decide the buy and sell signals based on the indicator RSI and Bollinger Bands.
def algo_trading(close_df): 
   
    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(close_df)

    # Concatenate the Bollinger Bands to the DataFrame
    bb_df = pd.concat([close_df, bbands_df], axis=1)

    # Set the Signal column
    bb_df["Signal"] = 0.0
#RSI calculation
    bb_df['RSI'] = ta.RSI(bb_df['close'], 14)
# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm 
    for index, row in bb_df.iterrows():
        if row["close"] < row["BB_LOWER"] and row["RSI"] < row["BB_LOWER"]:
            bb_df.loc[index, "Signal"] = 1.0
        if row["close"] > row["BB_UPPER"] and row["RSI"] > row["BB_UPPER"]:
            bb_df.loc[index,"Signal"] = -1.0
    return bb_df


In [163]:
# Visualize entry position relative to close price
def visualize_algo_trading(bb_df): 

    entry = bb_df[bb_df["Signal"] == 1.0]["close"].hvplot.scatter(
        color="green",
        marker="^",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize exit position relative to close price
    exit = bb_df[FSLR_bb_df["Signal"] == -1.0]["close"].hvplot.scatter(
        color="red",
        marker="v",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = bb_df[["close"]].hvplot(
        line_color="gray",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_upper = bb_df[["BB_UPPER"]].hvplot(
        line_color="purple",
        ylabel="Price in $",
        width=1000,
        height=400
    )


    bb_middle = bb_df[["BB_MIDDLE"]].hvplot(
        line_color="orange",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_lower = bb_df[["BB_LOWER"]].hvplot(
        line_color="blue",
        ylabel="Price in $",
        width=1000,
        height=400
    )


# Overlay plots
    bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
    return  bbands_plot

In [164]:
#function to calculate returns based on initial capital and signals establised from the trading algorithm. 
def cal_return(initial_capital,bb_df):
   
    # Set the share size
    share_size = 100

    # Buy a 100 share position when the dual moving average crossover Signal equals 1
    # Otherwise, `Position` should be zero (sell)
    bb_df['Position'] = share_size * bb_df['Signal']

    # Determine the points in time where a 100 share position is bought or sold
    bb_df['Entry/Exit Position'] = bb_df['Position'].diff()

    # Multiply the close price by the number of shares held, or the Position
    bb_df['Portfolio Holdings'] = bb_df['close'] * bb_df['Position']

    # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
    bb_df['Portfolio Cash'] = initial_capital - (bb_df['close'] * bb_df['Entry/Exit Position']).cumsum() 

    # Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
    bb_df['Portfolio Total'] = bb_df['Portfolio Cash'] + bb_df['Portfolio Holdings']

    # Calculate the portfolio daily returns
    bb_df['Portfolio Daily Returns'] = bb_df['Portfolio Total'].pct_change()

    # Calculate the portfolio cumulative returns
    bb_df['Portfolio Cumulative Returns'] = (1 + bb_df['Portfolio Daily Returns']).cumprod() - 1


    return bb_df

## First Solar Algorithm 

In [165]:

capital = 1e5
FSLR_bb_df = algo_trading(tickers_data['FSLR'])
plot_tr=visualize_algo_trading(FSLR_bb_df)   
FSLR_bb_df = cal_return(capital,FSLR_bb_df)


In [166]:
print(FSLR_bb_df['Portfolio Total'][-1])
plot_tr

104078.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Solar Edge Technologies

In [167]:

capital = 1e5
SEDG_bb_df = algo_trading(tickers_data['SEDG'])
plot_tr=visualize_algo_trading(SEDG_bb_df)   
SEDG_bb_df = cal_return(capital,SEDG_bb_df)


In [168]:
print(SEDG_bb_df['Portfolio Total'][-1])
plot_tr

99902.5


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Daqo New Energy Corp.

In [169]:
capital = 1e5
DQ_bb_df = algo_trading(tickers_data['DQ'])
plot_tr=visualize_algo_trading(DQ_bb_df)   
DQ_bb_df = cal_return(capital,DQ_bb_df)


In [170]:
print(DQ_bb_df['Portfolio Total'][-1])
plot_tr

100616.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Canadian Solar INC 

In [171]:
capital = 1e5
CSIQ_bb_df = algo_trading(tickers_data['CSIQ'])
plot_tr=visualize_algo_trading(CSIQ_bb_df)   
CSIQ_bb_df = cal_return(capital,CSIQ_bb_df)


In [172]:
print(CSIQ_bb_df['Portfolio Total'][-1])
plot_tr

100361.73


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Algonquin Power & Utilities Corp. 

In [173]:
capital = 1e5
AQN_bb_df = algo_trading(tickers_data['AQN'])
plot_tr=visualize_algo_trading(AQN_bb_df)   
AQN_bb_df = cal_return(capital,AQN_bb_df)


In [174]:
print(AQN_bb_df['Portfolio Total'][-1])
plot_tr

99823.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## SPY

In [175]:
capital = 1e5
SPY_bb_df = algo_trading(tickers_data['SPY'])
plot_tr=visualize_algo_trading(SPY_bb_df)   
SPY_bb_df = cal_return(capital,SPY_bb_df)


In [176]:
print(SPY_bb_df['Portfolio Total'][-1])
plot_tr

106960.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Green Stocks Portfolio

In [177]:
tickers_data.drop(columns='SPY',inplace=True)


In [178]:
tickers_data.columns=tickers_data.columns.droplevel(0)

,open,high,low,close,open,high,low,close,open,high,low,close,open,high,low,close,open,high,low,close
2019-01-02,41.63,43.1000,41.410,42.56,34.41,36.4450,34.3500,36.15,23.01,24.4100,22.540,24.05,14.05,14.76,14.00,14.44,10.01,10.030,9.9000,9.96
2019-01-03,43.97,44.6316,42.700,43.44,35.68,37.0678,35.5000,35.50,23.82,24.1014,23.230,23.30,15.02,15.27,14.80,15.14,9.99,10.090,9.9000,10.04
2019-01-04,43.98,46.6000,43.980,46.32,35.91,37.3800,35.2200,37.25,23.82,25.2100,23.620,24.74,15.43,16.53,15.20,16.37,10.10,10.210,10.0000,10.10
2019-01-07,46.46,47.7400,45.660,45.79,37.20,37.2000,34.9200,35.01,25.05,26.9300,24.885,26.56,16.06,17.40,15.96,16.80,10.19,10.270,10.1300,10.26
2019-01-08,46.80,46.9999,45.350,46.23,35.36,35.7366,34.0500,35.00,26.93,27.3400,25.830,26.05,16.98,17.42,16.82,17.13,10.30,10.390,10.2700,10.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-09,70.93,71.8600,69.770,69.83,296.14,302.0000,291.3400,291.67,59.90,63.2900,59.130,60.99,34.21,35.04,33.96,34.02,14.57,14.635,14.2800,14.30
2022-06-10,68.23,68.9500,67.070,68.00,286.56,292.0000,278.5800,284.36,60.00,65.3700,60.000,62.22,33.17,33.85,32.32,32.34,14.15,14.245,14.0525,14.18
2022-06-13,65.14,66.0600,62.770,63.13,269.47,271.5400,255.8250,264.34,58.81,59.9700,55.680,58.43,30.73,31.26,29.50,30.16,13.99,14.010,13.7250,13.82
2022-06-14,63.50,64.1500,61.075,61.95,265.33,269.5600,260.4201,264.63,59.71,59.8400,55.420,56.57,30.43,30.57,29.14,29.37,13.78,13.790,13.1500,13.19


In [180]:
#portfolio based on equal weight to each stock


tickers_data['Portfolio_open'] = capital + tickers_data['open'].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_high'] = capital + tickers_data['high'].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_low'] = capital + tickers_data["low"].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_close'] = capital + tickers_data["close"].pct_change().cumsum().mean(axis = 1) *capital
portfolio_df=pd.concat([tickers_data['Portfolio_open'],tickers_data['Portfolio_high'],tickers_data['Portfolio_low'], tickers_data['Portfolio_close']], keys=['open','high','low','close'], axis=1)
portfolio_df.dropna(inplace=True)
Portfolio_bb_df = algo_trading(portfolio_df)
plot_tr=visualize_algo_trading(Portfolio_bb_df)   
Portfolio_bb_df = cal_return(capital,Portfolio_bb_df)


C:\Users\amany\Anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [181]:
Portfolio_bb_df['Portfolio Total']
plot_tr

:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [151]:
print(Portfolio_bb_df['Portfolio Total'][-1])

2816781.599228788
